# Um modelo matemático para otimizar a sua vida na universidade

## Prof. Mayron César O. Moreira 

**Universidade Federal de Lavras (UFLA)**  
**Departamento de Ciência da Computação**  
**Lavras, Minas Gerais, Brasil**  

*III Semana da Matemática da UFLA (III SEMAT/UFLA)*  
*Lavras, MG, Brasil*

## Descrição do Problema

A vida acadêmica de um estudante é repleta de atividades. Disciplinas, grupos de estudo, iniciação científica, estágio, e por último (mas não menos importante), atividades que envolvam esportes e lazer. Mas como se programar para realizar tantas tarefas, mantendo uma boa qualidade de vida? Nesse curso, desenvolveremos um modelo matemático que servirá como uma boa ferramenta para auxiliar na otimização de seu tempo e de sua vida universitária!

### Tabela de Horários

Por meio de uma planilha, complete sua tabela de horários com as atividades fixas de sua semana. O arquivo se chama **"Horarios.xlsx"**, e está no diretório do *notebook* desse curso. Alguns exemplos: $(i)$ os horários de suas disciplinas presenciais; $(ii)$ horários de atividades esportivas ou de recreação (natação, futebol, dança de salão, etc); $(iii)$ reuniões de grupos de estudo.  

Vamos carregar as suas informações através de uma biblioteca do Python denominada *Pandas*. Assim, conseguiremos preencher os parâmetros do modelo matemático. Para padronizar os dados de entrada, insira a o símbolo **D-** antes de todas as disciplinas. Exemplo: **D-GCC218**, ou **D-01**.  

In [89]:
import pandas as pd; 
planilha1 = pd.read_excel('Horarios.xlsx', sheet_name='horario'); 
dados1 = planilha1.to_records(index=False)
    
planilha2 = pd.read_excel('Horarios.xlsx', sheet_name='carga')
dados2 = planilha2.to_records(index=False)
    
planilha3 = pd.read_excel('Horarios.xlsx', sheet_name='dias-atividade')
dados3 = planilha3.to_records(index=False)

### Contexto modelado

Nosso planejamento leva em conta os sete dias da semana, representados pelo conjunto $T$. Vamos considerar um conjunto de atividades $A$, divididos em um conjunto de disciplinas $D$ e um conjunto de atividades extras $E$ ($A = D \cup E$ e $D \cap E = \varnothing$). Os dias correspondentes a cada atividade é determinado pelo conjunto $T_a \subseteq T, a \in A$. Além disso, a carga horária semanal de $a \in A$ é dada por $w_a \in \mathbb{R}_+$. **Lembre-se:** a carga horária de uma disciplina $d \in D$ corresponde às horas dentro e fora de sala de aula. Utilizamos também um conjunto de *slots* de tempo $S = \{7, 8, ..., 22\}$. Desejamos determinar um plano de atividades semanais tal que:  

* **Restrição 1**: as atividades pré-fixadas devem ter seus horários respeitados;  
* **Restrição 2**: cada horário (*slot*) deve ser ocupado por uma atividade (garante que não haja sobreposição de horários);  
* **Restrição 3**: a carga horária das atividades deve ser cumprida;  
* **Restrição 4**: os sábados à partir das 18h devem ser dedicados ao **lazer**;    
* **Restrição 5**: de segunda à sexta, estabelecemos que hajam no máximo 2h de dedicação ao esporte por dia;  
* **Restrição 6**: as atividades de esporte deverão ser finalizadas até às 11h, de segunda à sexta;  

O objetivo de nosso modelo é obedecer todas as restrições e especificidades de nosso contexto, **maximizando** a carga horária de atividades de lazer! =)

### Parâmatros (*inputs*)

Façamos um resumo dos parâmetros necessários ao nosso modelo.  

* $D$: conjunto de disciplinas;  
* $E$: conjunto de atividades extras; 
* $A = D \cup E$: conjunto de atividades;  
* $\tilde{A}$: conjunto de triplas $(\overline{a},\overline{t},\overline{s})$ que correspondem a designações de atividades, hora e dia já pré-determinados;  
* $T$: dias da semana;  
* $S$: *slots* de tempo;  
* $T_a$: dias em que a atividade $a \in A$ deve ser realizada;  
* $w_a$: carga horária da atividade $a \in A$.  

Em Python, a implementação desses dados é apresentada abaixo, utilizando as informações preenchidas pela planilha 

In [90]:
D = [x[0] for x in dados2 if 'D-' in x[0]]
E = [x[0] for x in dados2 if 'D-' not in x[0]]
E = E + ['Lazer'] # Acrescentando a atividade Lazer
A = D + E
T = [x for x in dados1[0] if x != 'Hora']
S = [x[0] for x in dados1 if x[0] != 'Hora']

# Criando um dicionário de atividade:carga horária
w = { x[0]:x[1] for x in dados2 }

# Criando a lista de dias fixos para cada atividades
linhaDias = dados1[0]
ativDiaHora = { a:[] for a in A } # Conjunto \tilde{A}
ativDia = { a:[] for a in A }
diaAtivHora = { d:[] for d in linhaDias if d != 'Hora'}
alimentacao = [] # Conjunto de dias e horários que iremos nos alimentar (pausa)
for a in A:
    for i in range(1,len(dados1)):
        for j in range(len(dados1[i])):
            if(a == dados1[i][j]):
                ativDiaHora[a].append((linhaDias[j],dados1[i][0]))
                
                if(linhaDias[j] not in ativDia[a]):
                    ativDia[a].append(linhaDias[j])
                
                diaAtivHora[linhaDias[j]].append((a,dados1[i][0]))
                
            elif((dados1[i][j] == 'Almoço' or dados1[i][j] == 'Janta')
                 and ((linhaDias[j], dados1[i][0]) not in alimentacao)):
                alimentacao.append((linhaDias[j], dados1[i][0]))
                
# Calculando as possibilidades de dias para cada atividade
Ta = { a:[] for a in A }
for x in dados3:
    for y in x:
        if(y in T):
            Ta[x[0]].append(y)

### Modelo Matemático

Para modelar nosso problema matematicamente, utilizaremos a biblioteca PuLP do Python 3, declarada na sequência.

#### Variáveis de decisão

Vamos definir o conjunto de variáveis binárias $x_{ast} \in \{0,1\}$ igual a 1 se e somente a atividade $a \in A$ for alocada no slot de tempo $s \in S$, no dia $t \in T$.

#### Modelo matemático

Criando o modelo de **maximização**.

#### Função objetivo

Como cada *slot* de tempo é discretizado de hora em hora, nossa função objetivo consiste em maximizar a quantidade de horas que a variável $x$ relativa ao *lazer* aparece.

\begin{equation}
\max \sum_{s \in S}\sum_{t \in T}x_{\overline{a}st}
\end{equation}

em que $\overline{a}$ = Lazer.

#### Restrições 1: fixação de atividades definidas *a priori*

Seja a tripla $(\overline{a}, \overline{s}, \overline{t}) \in \tilde{A}$ como uma tripla de atividade, horário e dia de uma atividade fixa pertencentes a um conjunto de designações feitas *a priori*. Assim, nossa restrição pode ser escrita da seguinte forma: 

\begin{equation}
x_{\overline{a}\,\overline{s}\,\overline{t}} = 1, \forall (\overline{a},\overline{s},\overline{t}) \in \tilde{A}
\end{equation}

#### Restrições 2: cada *slot* de tempo deve ser ocupado por alguma atividade

\begin{equation}
\sum_{a \in A} x_{ast} = 1, \forall s \in S, \forall t \in T
\end{equation}

#### Restrições 3: a carga horária de todas as atividades deve ser cumprida (a exceção do 'Lazer', que deve ser maximizado)

\begin{equation}
\sum_{s \in S}\sum_{t \in T_a} x_{ast} = w_a, \forall a \in A \backslash \{\mbox{'Lazer'}\}
\end{equation}

#### Restrições 4: os sábados à partir das 18h devem ser dedicados ao **lazer**

\begin{equation}
x_{\tilde{a}s\tilde{t}} = 1, \forall s \in S, s \ge 18, \tilde{a} = \mbox{'Lazer'}, \tilde{t} = \mbox{'Sab'}
\end{equation}

18
19
20
21
22


#### Restrições 5: de segunda à sexta, estabelecemos que hajam no máximo 2h de dedicação ao esporte por dia. 

\begin{equation}
\sum_{s \in S}x_{\mbox{'Esp'},s,t} \le 2, \forall t \in T\backslash\{\mbox{'Sab','Dom'}\}
\end{equation}

#### Restrições 6: as atividades de esporte deverão ser finalizadas até às 11h, de segunda à sexta.

\begin{equation}
\sum_{s \in S, s \ge 11}\sum_{t \in T}x_{\mbox{'Esp'},s,t} = 0
\end{equation}

#### Resolvendo o modelo

In [101]:
modelo.solve()

1

#### Status da solução encontrada

In [102]:
print("Status da resolução do modelo:", LpStatus[modelo.status])

Status da resolução do modelo: Optimal


#### Total de horas de lazer

In [103]:
print("Horas de Lazer = ", value(modelo.objective))

Horas de Lazer =  28.0


#### Gerando sua planilha de horários

In [104]:
# Create a Pandas dataframe from some data.

Horarios = { (s,t):' ' for s in S for t in T }
Horarios['7','Seg'] = 'D-01'

for a in A:
    for s in S:
        for t in T:
            if(value(x[a][s][t]) >= 0.9):
                Horarios[s,t] = a

dicionarioHoras = { 'Horarios': S }
dicionarioHoras1 = { t:[Horarios[s,t] for s in S] for t in T }

z = {**dicionarioHoras, **dicionarioHoras1}
df = pd.DataFrame(z)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Resultado1.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Horários', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

### Novas restrições

Vamos aprimorar a solução desse modelo matemático. Consideremos as seguintes restrições adicionais:

* **Restrição 7**: as horas de estudo devem ser contínuas (sequenciais);  
* **Restrição 8**: os horários de Iniciação Científica devem ter períodos de no mínimo 2h contínuas;  
* **Restrição 9**: atividades esportivas não devem aparecer entre o estudo de disciplinas e entre IC;  
* **Restrição 10**: ao menos 2h fora de sala de aula deve ser gasta antes de cada aula presencial da disciplina $d \in D$.

#### Restrições 7: as horas de estudo devem ser contínuas (sequenciais)


#### Restrições 8: os horários de Iniciação Científica devem ter períodos de no mínimo 2h contínuas


#### Restrições 9: atividades esportivas não devem aparecer entre o estudo de disciplinas e entre IC.  


#### Restrições 10: ao menos 1h fora de sala de aula deve ser gasta antes de cada aula presencial da disciplina $d \in D$. 

Seja $\tilde{T}^a_{t_1,t_2}$ o conjunto de dias entre os dias $t_1$ e $t_2$ que atividade $a \in A$ pode ocorrer. Considere $preT(\overline{a})$ como o dia anterior ao que $\overline{a}$ é realizado, e $preS(\overline{a})$ como a hora que executamos a atividade $a$ em um dia anterior ao que $\overline{a}$ ocorre. Implementamos a definição desses conjuntos abaixo.

In [43]:
# Definição de $\tilde{T}^a_{t_1,t_2}$
def diasAtividadeIntervalo(a, t1, t2):
    i = T.index(t1)
    j = T.index(t2)
    Dias = []
    if(i <= j):
        Dias = T[(i+1):j]
    else:
        Dias = [t for t in T if not (j <= T.index(t) and T.index(t) <= i)]

    return [t for t in Dias if t in Ta[a]]
    
# Define o dia em que a atividade 'a' é executada antes do dia t
def preT(a, t):
    # Dia anterior a t
    if(ativDia[a].index(t) > 0):
        return ativDia[a][(ativDia[a].index(t) - 1)]
    else:
        return ativDia[a][(len(ativDia[a]) - 1)]
    
# Define o slot em que a atividade 'a' é executada antes do dia t
def preS(a, t):
    # Dia anterior a t
    if(ativDia[a].index(t) > 0):
        d = ativDia[a][(ativDia[a].index(t) - 1)]
    else:
        d = ativDia[a][(len(ativDia[a]) - 1)]

    for (a1, s) in diaAtivHora[d]:
        if(a1 == a):
            return s

#### Resolvendo o modelo

In [45]:
modelo.solve()

1

#### Status da solução encontrada

In [46]:
print("Status da resolução do modelo:", LpStatus[modelo.status])

Status da resolução do modelo: Optimal


#### Total de horas de lazer

In [47]:
print("Horas de Lazer = ", value(modelo.objective))

Horas de Lazer =  28.0


#### Gerando sua planilha de horários

In [48]:
# Create a Pandas dataframe from some data.

Horarios = { (s,t):' ' for s in S for t in T }
Horarios['7','Seg'] = 'D-01'

for a in A:
    for s in S:
        for t in T:
            if(value(x[a][s][t]) >= 0.9):
                Horarios[s,t] = a

dicionarioHoras = { 'Horarios': S }
dicionarioHoras1 = { t:[Horarios[s,t] for s in S] for t in T }

z = {**dicionarioHoras, **dicionarioHoras1}
df = pd.DataFrame(z)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Resultado2.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Horários', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

#### Enjoy it! =)